In [ ]:
import json
from pathlib import Path
import os

# 파일 경로 설정 (상대 경로)
current_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd()
project_root = current_dir.parent
IN_PATH = project_root / "data" / "raw" / "youth_policies_api.json"
OUT_PATH = project_root / "data" / "processed" / "youth_policies_filtered_kr_revised.json"


# 코드 -> 한글 매핑 딕셔너리들
PVSN_INST_GROUP = {
    "0054001": "중앙부처",
    "0054002": "지자체"
}

PLCY_PVSN_METHOD = {
    "0042001": "인프라 구축",
    "0042002": "프로그램",
    "0042003": "직접대출",
    "0042004": "공공기관",
    "0042005": "계약(위탁운영)",
    "0042006": "보조금",
    "0042007": "대출보증",
    "0042008": "공적보험",
    "0042009": "조세지출",
    "0042010": "바우처",
    "0042011": "정보제공",
    "0042012": "경제적 규제",
    "0042013": "기타"
}

PLCY_APPROVAL_STATUS = {
    "0044001": "신청",
    "0044002": "승인",
    "0044003": "반려",
    "0044004": "임시저장"
}

APPLY_PERIOD_TYPE = {
    "0057001": "특정기간",
    "0057002": "상시",
    "0057003": "마감"
}

BIZ_PERIOD_TYPE = {
    "0056001": "특정기간",
    "0056002": "상시",
    "0056003": "마감"
}

MARRIAGE_STATUS = {
    "0055001": "기혼",
    "0055002": "미혼",
    "0055003": "제한없음"
}

INCOME_CONDITION = {
    "0043001": "무관",
    "0043002": "연소득",
    "0043003": "기타"
}

MAJOR_REQUIREMENT = {
    "0011001": "인문계열",
    "0011002": "사회계열",
    "0011003": "상경계열",
    "0011004": "이학계열",
    "0011005": "공학계열",
    "0011006": "예체능계열",
    "0011007": "농산업계열",
    "0011008": "기타",
    "0011009": "제한없음"
}

JOB_REQUIREMENT = {
    "0013001": "재직자",
    "0013002": "자영업자",
    "0013003": "미취업자",
    "0013004": "프리랜서",
    "0013005": "일용근로자",
    "0013006": "(예비)창업자",
    "0013007": "단기근로자",
    "0013008": "영농종사자",
    "0013009": "기타",
    "0013010": "제한없음"
}

SCHOOL_REQUIREMENT = {
    "0049001": "고졸 미만",
    "0049002": "고교 재학",
    "0049003": "고졸 예정",
    "0049004": "고교 졸업",
    "0049005": "대학 재학",
    "0049006": "대졸 예정",
    "0049007": "대학 졸업",
    "0049008": "석·박사",
    "0049009": "기타",
    "0049010": "제한없음"
}

SPECIAL_REQUIREMENT = {
    "0014001": "중소기업",
    "0014002": "여성",
    "0014003": "기초생활수급자",
    "0014004": "한부모가정",
    "0014005": "장애인",
    "0014006": "농업인",
    "0014007": "군인",
    "0014008": "지역인재",
    "0014009": "기타",
    "0014010": "제한없음"
}


# 원본 필드명 -> 전처리 결과(한글) 필드명 매핑
FIELD_MAP = {
    # 기본 정보
    "plcyNm": "정책명",
    "plcyKywdNm": "정책키워드",
    "plcyExplnCn": "정책설명",
    "lclsfNm": "대분류",
    "mclsfNm": "중분류",

    # 지원 내용·조건
    "plcySprtCn": "지원내용",
    "earnMinAmt": "최소지원금액",
    "earnMaxAmt": "최대지원금액",
    "earnEtcCn": "기타지원조건",
    "sprtTrgtMinAge": "지원최소연령",
    "sprtTrgtMaxAge": "지원최대연령",

    # 기관 정보
    "sprvsnInstCdNm": "주관기관명",
    "operInstCdNm": "운영기관명",
    "rgtrInstCdNm": "등록기관명",
    "rgtrUpInstCdNm": "상위기관명",
    "rgtrHghrkInstCdNm": "상위등록기관명",

    # 신청·사업 기간/방법
    "aplyYmd": "신청기간",
    "plcyAplyMthdCn": "신청방법",
    "sbmsnDcmntCn": "제출서류",
    "bizPrdBgngYmd": "사업시작일",
    "bizPrdEndYmd": "사업종료일",

    # 심사·선정
    "srngMthdCn": "선정방법",

    # URL
    "aplyUrlAddr": "신청URL",
    "refUrlAddr1": "참고URL1",
    "refUrlAddr2": "참고URL2",

    # 기타 텍스트 조건
    "addAplyQlfcCndCn": "추가자격조건",
    "ptcpPrpTrgtCn": "참여제외대상",
}


def decode_code_to_korean(src: dict) -> dict:
    """
    코드 값을 한글로 변환 (코드 필드는 저장하지 않고 한글 필드만 반환)
    """
    decoded = {}
    
    # 재공기관그룹
    if src.get("pvsnInstGroupCd"):
        decoded["재공기관그룹"] = PVSN_INST_GROUP.get(src["pvsnInstGroupCd"], src["pvsnInstGroupCd"])
    
    # 정책제공방법
    if src.get("plcyPvsnMthdCd"):
        decoded["정책제공방법"] = PLCY_PVSN_METHOD.get(src["plcyPvsnMthdCd"], src["plcyPvsnMthdCd"])
    
    # 정책승인상태
    if src.get("plcyAprvSttsCd"):
        decoded["정책승인상태"] = PLCY_APPROVAL_STATUS.get(src["plcyAprvSttsCd"], src["plcyAprvSttsCd"])
    
    # 신청기간구분
    if src.get("aplyPrdSeCd"):
        decoded["신청기간구분"] = APPLY_PERIOD_TYPE.get(src["aplyPrdSeCd"], src["aplyPrdSeCd"])
    
    # 사업기간구분
    if src.get("bizPrdSeCd"):
        decoded["사업기간구분"] = BIZ_PERIOD_TYPE.get(src["bizPrdSeCd"], src["bizPrdSeCd"])
    
    # 혼인상태
    if src.get("mrgSttsCd"):
        decoded["혼인상태"] = MARRIAGE_STATUS.get(src["mrgSttsCd"], src["mrgSttsCd"])
    
    # 소득조건
    if src.get("earnCndSeCd"):
        decoded["소득조건"] = INCOME_CONDITION.get(src["earnCndSeCd"], src["earnCndSeCd"])
    
    # 전공요건
    if src.get("plcyMajorCd"):
        decoded["전공요건"] = MAJOR_REQUIREMENT.get(src["plcyMajorCd"], src["plcyMajorCd"])
    
    # 취업요건
    if src.get("jobCd"):
        decoded["취업상태"] = JOB_REQUIREMENT.get(src["jobCd"], src["jobCd"])
    
    # 학력요건
    if src.get("schoolCd"):
        decoded["학력요건"] = SCHOOL_REQUIREMENT.get(src["schoolCd"], src["schoolCd"])
    
    # 특화요건
    if src.get("sbizCd"):
        decoded["특화분야"] = SPECIAL_REQUIREMENT.get(src["sbizCd"], src["sbizCd"])
    
    return decoded


def transform_record(src: dict) -> dict:
    """
    원본 한 건(src)을 전처리 형식(dict)으로 변환.
    - FIELD_MAP에 정의된 항목만 남기고,
    - 값이 비어 있으면 생략
    - 코드 필드는 한글로 변환 후 코드는 삭제
    """
    dst = {}

    for src_key, dst_key in FIELD_MAP.items():
        value = src.get(src_key)
        if value is None:
            continue

        if isinstance(value, str) and value.strip() == "":
            continue

        dst[dst_key] = value
    
    # 코드를 한글로 변환한 필드 추가 (코드 필드는 포함하지 않음)
    decoded_fields = decode_code_to_korean(src)
    dst.update(decoded_fields)

    return dst


def main():
    # 1) 원본 JSON 로드
    with IN_PATH.open("r", encoding="utf-8") as f:
        raw = json.load(f)

    # 2) 정책 리스트 꺼내기 (result > youthPolicyList)
    policies = raw["result"]["youthPolicyList"]

    # 3) 각 정책을 전처리 포맷으로 변환
    transformed = []
    for p in policies:
        rec = transform_record(p)
        transformed.append(rec)

    # 4) 리스트 형태로 저장
    with OUT_PATH.open("w", encoding="utf-8") as f:
        json.dump(transformed, f, ensure_ascii=False, indent=2)

    print(f"변환 완료: {len(transformed)}건 -> {OUT_PATH}")


if __name__ == "__main__":
    main()


변환 완료: 3000건 -> c:\Users\31799\Desktop\SKN20-3rd-1TEAM\data\processed\youth_policies_filtered_kr_revised.json
코드 필드 → 한글 변환 완료! (코드 필드는 삭제됨)
